In [1]:
import requests

### Register as new ***User***

In [2]:
body={
    'username': 'blabber',
    'password': 'blabber',
    'email': 'a@g.com'
}

In [3]:
response = requests.post(url='http://localhost:8000/api/v1/users/', json=body)

In [4]:
response.json()

{'username': ['A user with that username already exists.']}

### Define an API

In [2]:
class ContractAPI:
    def __init__(self, base_url):
        self.base_url = base_url
        
    def login(self, username, password):
        
        body={
            'username': username,
            'password': password
        }
        url = self.base_url + "/token/"
        self.tokens = requests.post(url, json=body).json()
        
    def get_books(self):
        url = self.base_url + "/books/"
        self.headers = {'Authorization': "Bearer %s" % self.tokens['access']}
        books = requests.get(url, headers=self.headers)
        return books.json()
    
    def create_contractual_party(self, legal_entity, repeat=0):
        body = {'legal_entity': legal_entity}
        url = self.base_url + "/contractual_parties/"
        response = requests.post(url, json=body, headers=self.headers)
        if response.status_code == 401:
            self.refresh()
            print ('Token expired and refreshed. Please try again.')
        else:
            return response.json()

    def create_contract(self, other_party_id, other_party_role):
        body = {
            'other_party_role': other_party_role,
            'other_party': other_party_id,
        }        
        url = self.base_url + "/contracts/"
        response = requests.post(url, json=body, headers=self.headers)
        if response.status_code == 401:
            self.refresh()
            print ('Token expired and refreshed. Please try again.')
            return response.json()
        
        if response.status_code != 201:
            print(response)
            print(response.json())
            return response
        else:
            return response.json()
        

    def refresh(self):
        body={'refresh': self.tokens['refresh']}
        url = self.base_url + "/token/refresh/"
        response = requests.post(url, json=body)
        self.tokens['access'] = response.json()['access']
        

In [3]:
u_api = ContractAPI('http://localhost:8000/api/v1')
a_api = ContractAPI('http://localhost:8000/api/v1')

u_api.login('blubber', 'blubber')
a_api.login('blabber', 'blabber')

In [4]:
u_api.get_books()

[{'title': 'When doves fly',
  'subtitle': "it's raining shit",
  'author': 'Bernhardt Giersche',
  'isbn': '1234567812345',
  'price': '39.90'}]

In [5]:
a_api.get_books()

[{'title': 'When doves fly',
  'subtitle': "it's raining shit",
  'author': 'Bernhardt Giersche',
  'isbn': '1234567812345',
  'price': '39.90'}]

### Create a ***ContractualParty*** entry

In [6]:
response = a_api.create_contractual_party(legal_entity="Advertima")
a_cp_id=response['id']

In [7]:
response = u_api.create_contractual_party(legal_entity="Migros")
u_cp_id=response['id']

In [8]:
a_cp_id, u_cp_id

(51, 52)

### Create a ***Contract***

In [9]:
a_api.create_contract(other_party_id=a_cp_id, other_party_role='Publisher')

{'other_party_role': 'Publisher', 'other_party': 51, 'id': 23}

In [10]:
u_api.create_contract(other_party_id=u_cp_id, other_party_role='Publisher')

{'other_party_role': 'Publisher', 'other_party': 52, 'id': 24}

### Try to create an illegal ***Contract***

In [11]:
response = u_api.create_contract(other_party_id=a_cp_id, other_party_role='Publisher')

<Response [403]>
{'detail': 'You are not an administrator of contractual party.'}
